# Manifold Sculpting

In [1]:
import matplotlib.pyplot as plt
import torch

In [5]:
k = 2
data = torch.Tensor([[1,1],[7,7],[9,9],[12,12], [5,5], [2,2]])
p = data[2]

data_size = torch.tensor(data.size())
m = data_size[0] # number of data points
p_size = data_size[1:] # size of the single datapoint
p_size0 = p_size - torch.ones_like(p_size) # to be used for indexing
print(m)
print(p_size)

tensor(6)
tensor([2])


### Compute distances

In [6]:
x2 = data * data
x2 = x2.sum(axis=1)
# print(x2)
data_t = torch.transpose(data, 0, 1) # pb in dim >i (es foto...), credo funzioni cmq
xx = data@data_t
# print(xx)
# print(x2.unsqueeze(dim=1))
dist = torch.sqrt( x2 - 2*xx + x2.unsqueeze(dim=1) )
print(dist)

tensor([[ 0.0000,  8.4853, 11.3137, 15.5563,  5.6569,  1.4142],
        [ 8.4853,  0.0000,  2.8284,  7.0711,  2.8284,  7.0711],
        [11.3137,  2.8284,  0.0000,  4.2426,  5.6569,  9.8995],
        [15.5563,  7.0711,  4.2426,  0.0000,  9.8995, 14.1421],
        [ 5.6569,  2.8284,  5.6569,  9.8995,  0.0000,  4.2426],
        [ 1.4142,  7.0711,  9.8995, 14.1421,  4.2426,  0.0000]])


In [38]:
# extract the neighbours
_, indices = torch.sort(dist)
print(indices)
#print(dist)
# keep the first k (without the point itself)
kneighb = indices[:, 1:k+1]
kdist   = torch.zeros_like(kneighb)

a = []
for i in range(m):
    a.append( dist[kneighb[i][:], i] )
print(a)
kdist = torch.reshape( torch.cat( a, 0 ), (-1, k) )
del a

print(kdist)

tensor([[0, 5, 4, 1, 2, 3],
        [1, 2, 4, 3, 5, 0],
        [2, 1, 3, 4, 5, 0],
        [3, 2, 1, 4, 5, 0],
        [4, 1, 5, 0, 2, 3],
        [5, 0, 4, 1, 2, 3]])
[tensor([1.4142, 5.6569]), tensor([2.8284, 2.8284]), tensor([2.8284, 4.2426]), tensor([4.2426, 7.0711]), tensor([2.8284, 4.2426]), tensor([1.4142, 4.2426])]
tensor([[1.4142, 5.6569],
        [2.8284, 2.8284],
        [2.8284, 4.2426],
        [4.2426, 7.0711],
        [2.8284, 4.2426],
        [1.4142, 4.2426]])
